# Run Scheduling Shift
Example (Shift Scheduling at a Retail Store).  
Consider a retail store that is open for business from 10:00 to 21:00.  
There are five shift patterns.  

| Pattern |  Hours to work | Total hours | Cost |
|:-------:|:--------------:|:-----------:|:----:|
|    1    | 10:00 to 18:00 |      8      |  50  |
|    2    | 13:00 to 21:00 |      8      |  60  |
|    3    | 12:00 to 18:00 |      6      |  30  |
|    4    | 10:00 to 13:00 |      3      |  15  |
|    5    | 18:00 to 21:00 |      3      |  13  |  

Staffing requirements at the store vary from hour to hour.  
  
|  Period  | Staffing Requirement |
|:--------:|:--------------------:|
| 10 to 11 |           3          |
| 11 to 12 |           4          |
| 12 to 13 |           6          |
| 13 to 14 |           4          |
| 14 to 15 |           7          |
| 15 to 16 |           8          |
| 16 to 17 |           7          |
| 17 to 18 |           6          |
| 18 to 19 |           4          |
| 19 to 20 |           7          |
| 20 to 21 |           8          |

In [1]:
# libraries
import pandas as pd
from model import Shifts

## Sets & Parameters

In [2]:
df_shifts = pd.read_csv('shifts.csv')
df_forecast = pd.read_csv('forecast.csv')
sets = {}
parameters = {}

sets['Forecast'] = df_forecast['staffing'].to_list()
sets['Shifts'] = df_shifts['pattern'].to_list()

parameters['Cost'] = df_shifts['cost'].to_list()

The problem can be formulated as an integer program with the following $c$
vector, $A$ matrix and $b$ vector:  
$$ c = (50,60,30,15,15) $$ 
$$ A = \begin{bmatrix}
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 0 & 1 & 0 \\
1 & 0 & 1 & 1 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
1 & 1 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 1 \\

\end{bmatrix} \quad 
b = \begin{bmatrix}
3 \\
4 \\
6 \\
4 \\
7 \\
8 \\
7 \\
6 \\
4 \\
7 \\
8 \\

\end{bmatrix}

## Create $A$ Matrix with shift & forecast data 

In [3]:
def create_shift_pattern(shifts: pd.DataFrame, forecast: pd.DataFrame):
    """Create Matrix A  shift coverage pattern"""
    A = [[0 for _ in range(len(shifts))] for __ in range(len(forecast))]
    for j, shift in shifts.iterrows():
        for i, demand in forecast.iterrows():
            if shift['period_in'] <= i < shift['period_out']:
                A[i][j] = 1
    return A

hour_period = {valor: clave for clave, valor in df_forecast['period'].to_dict().items()}
df_shifts['period_in'] = df_shifts['hour_init'].apply(lambda x: hour_period[x])
df_shifts['period_out'] = df_shifts['hour_end'].apply(lambda x: hour_period[x-1])


parameters['A'] = create_shift_pattern(df_shifts, df_forecast)

## Create Model

In [4]:
model = Shifts()
model.sets(sets)
model.parameters(parameters)
model.variables()
model.constrains()
model.objective()
result = model.execute()


Status: OptimizationStatus.OPTIMAL


In [6]:
result

{'W': 7, 'Shifts': {0: 0, 1: 6, 2: 1, 3: 0, 4: 0}}

## Results
Using the results obtained with the optimization problem, we can calculated the staffing requirement per shift and the total cost of staffing

In [21]:
df_result = pd.DataFrame(list(result['Shifts'].items()), columns=['pattern',"staff_requirement"])
df_result = pd.merge(df_shifts, df_result, on='pattern', how='left')
df_result['cost_staffing'] = df_result['staff_requirement'] * df_result['cost']

In [22]:
df_result

,pattern,hour_init,hour_end,duration,cost,period_in,period_out,staff_requirement,cost_staffing
0,0,10,18,8,50,0,7,0,0
1,1,13,21,8,60,3,10,6,360
2,2,12,18,6,30,2,7,1,30
3,3,10,13,3,15,0,2,0,0
4,4,18,21,3,13,8,10,0,0


In [23]:
staff_cost = df_result['cost_staffing'].sum()
total_workforce = df_result['staff_requirement'].sum()
print(f"Total cost Staffing: {staff_cost}")
print(f"Total Workforce: {total_workforce}")

Total cost Staffing: 390
Total Workforce: 7


With this algorithm can minimize the cost of the personal without affecting the staffing requiremet